# Gradio Interface to Chat 

This is the final product showcasing the CRM AI Assistant built in the Gradio environment. The current version uses DummyCRM, a test module that simulates the interface and functionality for demonstration purposes.

Our CRM AI Assistant can:

* Generate intelligent business insights and pipeline analyses using advanced prompts, providing evaluations of conversion rates, deal performance, and strategic recommendation for sales optimization (TO-DO).

* Produce detailed Matplotlib/Seaborn charts visualizing opportunity pipeline stages, revenue distribution by industry, lead conversion rates, and deal amount distributions across the sales organization.

Advanced features will be implemented soon!

### Imports and Setup

In [4]:
import os
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import time

# from your_llm_module import CRMAssistant
# from your_search_module import indexer 

/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/nissiotoo/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load clean data
base_path = "data_directory/clean_data"
pipeline = pd.read_csv(os.path.join(base_path, "Pipeline.csv"))
accounts = pd.read_csv(os.path.join(base_path, "Accounts.csv"))
teams = pd.read_csv(os.path.join(base_path, "Teams.csv"))
products = pd.read_csv(os.path.join(base_path, "Products.csv"))

#### Importing Account Summary Agent

In [ ]:
# from account_summary_agent import AccountSummaryAgent

SyntaxError: invalid decimal literal (3005585998.py, line 1)

In [ ]:
# Initialize the agent
# account_agent = AccountSummaryAgent(model_name="gemini-2.5-flash")

NameError: name 'AccountSummaryAgent' is not defined

#### Importing Email Drafting Agent

In [ ]:
# from email_drafting_agent import EmailDraftingAgent

ModuleNotFoundError: No module named 'email_drafting_agent'

In [ ]:
# Initialize the agent
# email_agent = EmailDraftingAgent(model_name="gemini-2.5-flash")

### CRM and Search Modules

In [ ]:
# actual CRM module/assistant to add

# crm = CRMAssistant(
#     accounts_df=accounts,
#     pipeline_df=pipeline,
#     teams_df=teams,
#     products_df=products
# )

#### CRM Class for Email Queries

In [ ]:
class CRM:
    def __init__(self, accounts_df, pipeline_df, teams_df, products_df):
        self.accounts = accounts_df
        self.pipeline = pipeline_df
        self.teams = teams_df
        self.products = products_df
        
        # Initialize agents
        from email_drafting_agent import EmailDraftingAgent
        self.email_agent = EmailDraftingAgent(model_name="gemini-2.5-flash")
    
    def process_query(self, query, context=None):
        """
        Process user queries and route to appropriate agent.
        """
        query_lower = query.lower()
        
        # Email drafting patterns
        email_patterns = [
            'draft email', 'write email', 'compose email', 
            'send email', 'email draft', 'create email',
            'follow up', 'introduction email', 'proposal email'
        ]
        
        # Check if query is about email drafting
        if any(pattern in query_lower for pattern in email_patterns):
            return self._handle_email_query(query, context)
        
        # ... handle other query types (account summary, opportunities, etc.)
        
        return f"Received query: {query}"
    
    def _handle_email_query(self, query, context):
        """
        Handle email drafting requests.
        """
        try:
            # Determine email type from query
            email_type = self._detect_email_type(query)
            
            # Get account data from context or query
            account_data = self._extract_account_data(context)
            
            # Get opportunity data if available
            opportunity_data = self._extract_opportunity_data(context)
            
            # Detect tone
            tone = self._detect_tone(query)
            
            # Generate email
            email = self.email_agent.draft_email(
                email_type=email_type,
                account_data=account_data,
                opportunity_data=opportunity_data,
                tone=tone
            )
            
            return email
            
        except Exception as e:
            return f"Error generating email: {str(e)}"
    
    def _detect_email_type(self, query):
        """
        Detect the type of email from the query.
        """
        query_lower = query.lower()
        
        if 'follow' in query_lower or 'follow up' in query_lower:
            return 'follow_up'
        elif 'introduction' in query_lower or 'intro' in query_lower:
            return 'introduction'
        elif 'proposal' in query_lower:
            return 'proposal'
        elif 'check in' in query_lower or 'checking in' in query_lower:
            return 'check_in'
        elif 'closing' in query_lower or 'close deal' in query_lower:
            return 'closing'
        else:
            return 'follow_up'  # Default
    
    def _detect_tone(self, query):
        """
        Detect the desired tone from the query.
        """
        query_lower = query.lower()
        
        if 'friendly' in query_lower or 'casual' in query_lower:
            return 'friendly'
        elif 'urgent' in query_lower or 'asap' in query_lower:
            return 'urgent'
        elif 'consultative' in query_lower or 'advisory' in query_lower:
            return 'consultative'
        else:
            return 'professional'  # Default
    
    def _extract_account_data(self, context):
        """
        Extract account data from context.
        """
        if context and 'account' in context:
            account_name = context['account']
            account_row = self.accounts[self.accounts['account'] == account_name]
            
            if not account_row.empty:
                row = account_row.iloc[0]
                return {
                    'account_name': row['account'],
                    'sector': row['sector'],
                    'revenue': row['revenue'],
                    'employees': row['employees'],
                    'office_location': row['office_location']
                }
        
        # Return generic data if no specific account
        return {
            'account_name': 'Prospective Client',
            'sector': 'Technology',
            'revenue': 0,
            'employees': 0,
            'office_location': 'Unknown'
        }
    
    def _extract_opportunity_data(self, context):
        """
        Extract opportunity data from context.
        """
        if context and 'opportunity_id' in context:
            opp_id = context['opportunity_id']
            opp_row = self.pipeline[self.pipeline['opportunity_id'] == opp_id]
            
            if not opp_row.empty:
                row = opp_row.iloc[0]
                return {
                    'opportunity_id': row['opportunity_id'],
                    'product': row['product'],
                    'deal_stage': row['deal_stage'],
                    'close_value': row['close_value'],
                    'engage_date': str(row['engage_date']),
                    'close_date': str(row['close_date'])
                }
        
        return None

#### New Interface with Email Support

In [ ]:
def create_enhanced_interface():
    """
    Create Gradio interface with email drafting support.
    """
    
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        # CRM AI Assistant with Email Drafting
        ## Salesforce Team2B
        
        **Capabilities:**
        - 📊 Account Summaries
        - 💰 Opportunity Analysis  
        - ✉️ **AI-Powered Email Drafting**
        - 📈 Data Visualization
        """)
        
        with gr.Tabs():
            # Tab 1: General CRM Queries
            with gr.Tab("CRM Assistant"):
                gr.Markdown("### Ask questions about accounts, opportunities, or request insights")
                
                with gr.Row():
                    user_query = gr.Textbox(
                        label="Your Question",
                        placeholder="e.g., 'Give me a summary of Acme Corp' or 'Find high-value opportunities'",
                        lines=2
                    )
                
                with gr.Row():
                    account_filter = gr.Dropdown(
                        choices=["None"] + sorted(accounts["account"].unique().tolist()),
                        label="Filter by Account (optional)",
                        value="None"
                    )
                    agent_filter = gr.Dropdown(
                        choices=["None"] + sorted(pipeline["sales_agent"].unique().tolist()),
                        label="Filter by Sales Agent (optional)",
                        value="None"
                    )
                
                submit_btn = gr.Button("Get Answer", variant="primary")
                
                with gr.Row():
                    output = gr.Textbox(label="Response", lines=15)
                
                chart_output = gr.Plot(label="Visualization")
                
                submit_btn.click(
                    crm_chatbot_wrapper,
                    inputs=[user_query, account_filter, agent_filter],
                    outputs=[output, chart_output]
                )
            
            # Tab 2: Email Drafting
            with gr.Tab("Email Drafting"):
                gr.Markdown("""
                ### AI-Powered Email Generation
                Generate personalized sales emails based on account data and opportunity context.
                """)
                
                with gr.Row():
                    with gr.Column(scale=1):
                        email_type = gr.Dropdown(
                            choices=[
                                "introduction",
                                "follow_up", 
                                "proposal",
                                "check_in",
                                "closing"
                            ],
                            label="Email Type",
                            value="follow_up"
                        )
                        
                        email_account = gr.Dropdown(
                            choices=sorted(accounts["account"].unique().tolist()),
                            label="Select Account",
                            value=accounts["account"].iloc[0]
                        )
                        
                        email_tone = gr.Dropdown(
                            choices=["professional", "friendly", "urgent", "consultative"],
                            label="Tone",
                            value="professional"
                        )
                        
                        custom_instructions = gr.Textbox(
                            label="Additional Context (optional)",
                            placeholder="e.g., 'Mention our Q4 promotion' or 'Reference their budget concerns'",
                            lines=3
                        )
                        
                        generate_email_btn = gr.Button("Generate Email", variant="primary")
                    
                    with gr.Column(scale=2):
                        email_output = gr.Textbox(
                            label="Generated Email",
                            lines=20,
                            show_copy_button=True
                        )
                
                # Example queries
                gr.Markdown("""
                ### Example Requests:
                - "Draft a follow-up email for Acme Corp"
                - "Write an introduction email in a friendly tone"
                - "Create a proposal email with urgent tone"
                """)
                
                def generate_email_wrapper(email_type, account_name, tone, custom_context):
                    """
                    Wrapper function to generate emails from Gradio inputs.
                    """
                    try:
                        # Get account data
                        account_row = accounts[accounts['account'] == account_name].iloc[0]
                        account_data = {
                            'account_name': account_row['account'],
                            'sector': account_row['sector'],
                            'revenue': account_row['revenue'],
                            'employees': account_row['employees'],
                            'office_location': account_row['office_location']
                        }
                        
                        # Get opportunity data if exists
                        account_opps = pipeline[pipeline['account'] == account_name]
                        opportunity_data = None
                        
                        if not account_opps.empty:
                            latest_opp = account_opps.iloc[0]
                            opportunity_data = {
                                'product': latest_opp['product'],
                                'deal_stage': latest_opp['deal_stage'],
                                'close_value': latest_opp['close_value'],
                                'engage_date': str(latest_opp['engage_date'])
                            }
                        
                        # Generate email
                        email = email_agent.draft_email(
                            email_type=email_type,
                            account_data=account_data,
                            opportunity_data=opportunity_data,
                            custom_context=custom_context if custom_context else None,
                            tone=tone
                        )
                        
                        return email
                        
                    except Exception as e:
                        return f"Error generating email: {str(e)}"
                
                generate_email_btn.click(
                    generate_email_wrapper,
                    inputs=[email_type, email_account, email_tone, custom_instructions],
                    outputs=[email_output]
                )
        
        # Sample queries section
        gr.Markdown("""
        ---
        ### Sample Queries for CRM Assistant:
        - "Give me a summary of account Acme Corp"
        - "Find opportunities closing this month"
        - "Draft a follow-up email for [account name]"
        - "Show me high-value deals in the pipeline"
        """)
    
    return demo

### Data Visualization & Analytics

In [ ]:
# Task 3c: Data Visualization & Analytics (Starter)
def generate_chart(account=None, agent=None, timeframe=None):
    fig, ax = plt.subplots(figsize=(10, 11))
    df = pipeline.copy()

    if account:
        df = df[df["account"] == account]
    if agent:
        df = df[df["sales_agent"] == agent]
    if timeframe:
        df = df[df["engage_date"] >= timeframe]

    summary = df.groupby("deal_stage")["close_value"].sum()
    summary.plot(kind="bar", ax=ax)
    ax.set_title("Deal Value by Stage")
    ax.set_ylabel("Close Value ($)")
    ax.set_xlabel("Deal Stage")

    return fig

### Standalone Email Generation

In [ ]:
def quick_test_email():
    """
    Quick test function to verify email generation works.
    """
    print("Testing Email Drafting Agent...")
    
    test_account = {
        'account_name': 'Test Company Inc',
        'sector': 'Technology',
        'revenue': 5000000,
        'employees': 200,
        'office_location': 'San Francisco, CA'
    }
    
    test_opp = {
        'product': 'Enterprise Solution',
        'deal_stage': 'Negotiation',
        'close_value': 50000,
        'engage_date': '2024-11-01'
    }
    
    email = email_agent.draft_email(
        email_type='follow_up',
        account_data=test_account,
        opportunity_data=test_opp,
        tone='professional'
    )
    
    print("\n" + "="*80)
    print("GENERATED EMAIL:")
    print("="*80)
    print(email)
    print("="*80 + "\n")
    
    return email

In [ ]:
# Run test
quick_test_email()